In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


In [3]:
data=pd.read_csv('trollers.csv')

In [4]:
data=data.drop(columns=['mmsi','timestamp','source'])

In [5]:
data1 = data.copy()
data = data[data['is_fishing'] != -1]

In [6]:
data.shape

(7845, 7)

In [7]:
x=data.drop('is_fishing',axis=1)
y=data['is_fishing']

In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=997)

In [9]:
rfr = RandomForestRegressor()

In [10]:
y_train

84139     1.0
55112     0.0
84644     0.0
55264     0.0
4730      1.0
         ... 
147320    0.0
55041     0.0
55194     0.0
55009     1.0
150967    0.0
Name: is_fishing, Length: 6276, dtype: float64

In [11]:
rfr.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [12]:
fit = rfr.score(x_test, y_test)
print("R-squared:", fit) 

R-squared: 0.7918610106141969


In [13]:
y_pred = rfr.predict(x_test)
y_pred

array([1. , 0. , 0. , ..., 0.2, 0.8, 1. ])

In [14]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)
print("RMSE: ", mse*(1/2.0)) 

MSE:  0.04882931139037532
RMSE:  0.02441465569518766


In [15]:
new_data = x_test.copy()

In [16]:
new_data["Predicted"] = y_pred

In [17]:
new_data["Actual"] = y_test.to_list()

In [18]:
new_data["fishing_or_not"] = np.where(new_data.Predicted >= 0.2, 'Fishing', "Not_fishing")

In [19]:
new_data

,distance_from_shore,distance_from_port,speed,course,lat,lon,Predicted,Actual,fishing_or_not
85426,47852.769531,74731.023438,1.8,167.300003,47.459053,-124.777283,1.0,1.0,Fishing
53416,999.975464,1414.178833,0.0,0.000000,57.109634,12.241582,0.0,0.0,Not_fishing
53612,999.975464,1414.178833,0.0,0.000000,57.109619,12.241518,0.0,1.0,Not_fishing
84528,62167.597656,85443.796875,7.9,151.699997,45.864174,-124.547264,0.0,0.0,Not_fishing
150182,0.000000,5830.808594,0.0,0.000000,41.268188,36.381615,0.0,0.0,Not_fishing
...,...,...,...,...,...,...,...,...,...
148500,0.000000,5830.808594,0.0,0.000000,41.267944,36.381847,0.0,0.0,Not_fishing
150387,7070.894531,8602.114258,2.4,126.300003,41.345921,36.390285,0.5,0.0,Fishing
53917,999.975464,1414.178833,0.0,227.000000,57.109634,12.241550,0.2,0.0,Fishing
150606,11704.412109,12529.657227,2.5,118.900002,41.336842,36.449585,0.8,0.0,Fishing


In [20]:
def get_final_outut(fit, shore_dis, port_dist, speed, course, lat, lon):
    x_test = pd.DataFrame(columns= ["distance_from_shore", "distance_from_port", "speed", "course", "lat", "lon"])
    x_test["distance_from_shore"] = [shore_dis]
    x_test["distance_from_port"]= [port_dist]
    x_test["speed"]=[speed]
    x_test["course"]=[course]
    x_test["lat"]=[lat]
    x_test["lon"]=[lon]    
    
    y_pred = rfr.predict(x_test)
    
    fnf = "fishing" if y_pred >0.2 else "Not_fishing"
    
    return fnf

In [21]:
get_final_outut(fit,11704.412109,1414.178833,2.4,126.300003,38.239231,12.241550)

'fishing'

In [22]:
import pickle

In [23]:
pickle.dump(rfr, open('rfrfinal_prediction.pickle', 'wb'))

In [24]:
model = pickle.load(open('rfrfinal_prediction.pickle','rb'))
